In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import numpy as np
from pyspark.sql import SparkSession
%config Completer.use_jedi = False

In [2]:
# Load the dataset
df = pd.read_table('lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', lineterminator='\n', warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
df_profile = pd.read_csv('lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=True, skiprows=1, names=['user', 'gender', 'age', 'country', 'signup'])

In [3]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [4]:
# Get rows with song names containing multiple rows
duplicated = df.loc[df['song-id'].isin(grouped[grouped['song'] > 1].index)]
duplicated = duplicated.drop(columns=['user', 'timestamp', 'artist', 'artist-id'])
containis_extra_rows = duplicated.apply(lambda x: pd.Series({'id': x[0], 'song': x[1], 'flag':'\n' in x[1]}), axis=1)
containis_extra_rows = containis_extra_rows.loc[containis_extra_rows['flag']]

In [5]:
# Iterate over those 5k rows to get the extra rows and correct the song name
for index, row in containis_extra_rows.iterrows():
    row_break = row['song'].split('\n', 1)
    song_name = row_break[0]
    df.loc[df['song-id'] == row['id'], 'song'] = song_name
    
    tsv = StringIO(row_break[1])
    df_extra = pd.read_csv(tsv, sep="\t", warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
    df = df.append(df_extra, ignore_index=True)

### Fix names

In [6]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [7]:
# Check how many ids has more than 1 song name
grouped[grouped['song'] > 1].shape[0]

200

In [8]:
ids_to_be_fixed = grouped[grouped['song'] > 1].index

In [9]:
df = df.set_index('song-id')

In [10]:
map_to_fix = df.groupby(df.index)['song'].head(1)

In [11]:
df.loc[ids_to_be_fixed, 'song'] = df.loc[ids_to_be_fixed].reset_index()['song-id'].map(lambda x: map_to_fix[x])
df = df.reset_index()

### Create matrix

In [12]:
matrix_init = df.groupby(['user', 'song']).count()

In [13]:
matrix_init = matrix_init['timestamp']

In [14]:
matrix_init = matrix_init.reset_index()

In [15]:
del df

In [10]:
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

# Create PySpark DataFrame from Pandas
# sparkDF=spark.createDataFrame(matrix_init) 
# sparkDF.printSchema()
# sparkDF.show(5)


In [17]:
matrix = (sparkDF
            .groupby(sparkDF.song)
            .pivot("user")
            .avg("timestamp"))

In [18]:
# Save file local folder, delimiter by default is ,
matrix.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('lastfm-dataset-1K/my.csv')

In [19]:
matrix.repartition(1).write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('lastfm-dataset-1K/my2.csv')

In [21]:
matrix_pd = pd.read_csv('lastfm-dataset-1K/my2.csv/part-00000-9a28ec76-1292-49cd-904a-60a800e91bc2-c000.csv')

ParserError: Error tokenizing data. C error: Expected 993 fields in line 5341, saw 994


In [15]:
df_spark = spark.read.format("csv").load("lastfm-dataset-1K/my.csv")

In [17]:
df_spark.repartition(1).write.parquet("lastfm-dataset-1K/my2.parquet") 

In [21]:
import pyarrow.parquet as pq

In [22]:
arrow = pq.read_table('lastfm-dataset-1K/my2.parquet')

In [ ]:
matrix_pd = arrow.to_pandas()